### Add venv modules

In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import pandas as pd
import pint
import sys
from tqdm import tqdm
from joblib import dump, load

In [2]:
sys.path.append(str(Path.cwd().parent))

### Add local modules

In [3]:
from src.utils import safe_parse_quantity , calculate_thermal_entropy_generation
from comsol_module.comsol_classes import COMSOL_VTU
from comsol_module.entropy import calculate_S_therm

### Import 

In [4]:
ROOT = Path().cwd().parent
PARAMETER_SPACE = "07"
DATA_TYPE = "Test"
IS_LOAD_NPY : bool = True
EXPORT_FIELD = "Temperature"
IS_LOAD_JOBLIB : bool = False
IS_EXPORT = True
IS_LOAD_VTU = False # For entropy
assert IS_LOAD_NPY != IS_LOAD_JOBLIB
ROOT

PosixPath('/Users/thomassimader/Library/CloudStorage/OneDrive-geomeconGmbH/PhD/NIRB')

In [5]:
# export_folder = ROOT / "data" / PARAMETER_SPACE / f"{DATA_TYPE}Mapped" / "s100_100_100_b0_4000_0_5000_-4000_-0" / "Exports"
export_folder = ROOT / "data" / PARAMETER_SPACE / "Exports" / "VNC"
assert export_folder.exists()
import_folder = ROOT / "data" / PARAMETER_SPACE / f"{DATA_TYPE}OriginalVNC" #export_folder #ROOT / "data" / PARAMETER_SPACE / ""
# import_folder = export_folder
assert import_folder.exists()


In [6]:
if IS_LOAD_NPY:
    try:
        temperatures      = np.load(import_folder  / f"{DATA_TYPE}_{EXPORT_FIELD}.npy" )
    except FileNotFoundError:
        temperatures = None
    try:
        temperatures_diff = np.load(import_folder  / f"{DATA_TYPE}_{EXPORT_FIELD}_minus_tgrad.npy" )
    except FileNotFoundError:
        temperatures_diff = None
    try:
        sim_times_path = import_folder  / f"{DATA_TYPE}_sim_times.npy"
        sim_times = np.load(sim_times_path)
    except FileNotFoundError:
        sim_times = None
if IS_LOAD_JOBLIB:
    temperatures      = load(import_folder / f"{DATA_TYPE}_temperatures.joblib" )
    temperatures_diff = load(import_folder / f"{DATA_TYPE}_temperatures_diff.joblib" )

In [7]:
N_SNAPS = len(temperatures_diff)
colors = px.colors.sample_colorscale("jet", [n/(N_SNAPS) for n in range(N_SNAPS)])

In [8]:
# comsol_data  = COMSOL_VTU( ROOT / "data" / PARAMETER_SPACE / "Training" / f"{DATA_TYPE}_000.vtu")
# comsol_data  = COMSOL_VTU(import_folder / "Training_000_s50_50_50_b50_3950_50_4950_-3950_0.vti")
# comsol_data = COMSOL_VTU("/Users/thomassimader/Documents/NIRB/data/01/TrainingMapped/s100_100_100_b0_4000_0_5000_-4000_0/Training_000_s100_100_100_b0_4000_0_5000_-4000_0.vtu")
# comsol_data = COMSOL_VTU(ROOT / f"data/{PARAMETER_SPACE}/TrainingOriginal/Training_100.vtu")
comsol_data = COMSOL_VTU(import_folder / "Test_000.vtu")


## Plot

### Min max temperature difference

In [9]:
if temperatures_diff is not None:
    fig = go.Figure()
    for idx_snap, temperature_array in enumerate(temperatures_diff):
    # customdata=np.stack(([t_h] * len(comsol_data.times), [host_k] * len(comsol_data.times)) , axis=-1),
        input_args = {'x' : list(comsol_data.times.values()),
                    #   'customdata' : customdata,
                    #   'hovertemplate' : "Time: %{x:.2e}<br>Temperature: %{y:.2f}<br>T_h: %{customdata[0]:.2f}<br><br>host_k: %{customdata[1]:.2e}<br>",
                        'legendgroup' : f"{idx_snap:03d}",
                        'name' : f"{idx_snap:03d}",
                        'line' : dict(color=colors[idx_snap]),
                        'opacity': 0.4}
        fig.add_trace(go.Scatter(y=np.max(temperature_array, axis=1), **input_args)) 
        fig.add_trace(go.Scatter(y=np.min(temperature_array, axis=1), **input_args)) 

    fig.update_layout(
        xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
        yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
        showlegend=False,        # Hide the legend
        title_text=f'Temperature Difference (T-T0) - {DATA_TYPE} Snapshots - Parameter Space {PARAMETER_SPACE}', # title of plot
        xaxis_title="Time [s]",    # Optional: label for x-axis
        yaxis_title="Temperature [K]"     # Optional: label for y-axis
    )
    if IS_EXPORT:
        fig.write_html(export_folder / f"{DATA_TYPE}_MinMaxTemperatures.html")
        fig.write_image(export_folder / f"{DATA_TYPE}_MinMaxTemperatures.png")
    fig.show()


### Simulation Time

In [10]:
fig = go.Figure(data=[go.Histogram(
    x=sim_times / 60,
    histnorm='percent',
    name='control', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        # start=-4.0,
        # end=3.0,
        size=2,
        ),
    marker_color='#EB89B5',
    opacity=0.75,
     texttemplate="%{x}"
    )])

fig.update_layout(
    title_text=f'Simulation Time - {DATA_TYPE} Snapshots - Parameter Space {PARAMETER_SPACE}', # title of plot
    xaxis_title_text='Time [min]', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargroupgap=0.1, # gap between bars of the same location coordinates
)
if IS_EXPORT:
    fig.write_html(export_folder / f"{DATA_TYPE}_SimulationTime.html")
    fig.write_image(export_folder / f"{DATA_TYPE}_SimulationTime.png")
fig.show()


## Calculate Entropy


In [11]:
param_folder = ROOT / "data" / PARAMETER_SPACE / "Exports"
param_files = sorted([path for path in param_folder.rglob(f"{DATA_TYPE}*.csv")])
assert len(param_files) == N_SNAPS
ureg = pint.get_application_registry()

In [12]:
IS_USE_TGRAD = False
if PARAMETER_SPACE == "03":
    IS_USE_TGRAD = True

In [13]:
comsol_data_ref = comsol_data

if not IS_LOAD_VTU:
    N_SNAPS = len(temperatures)
    N_TIME_STEPS = len(comsol_data_ref.times)
    entropy_gen_unit = np.zeros((N_SNAPS, N_TIME_STEPS))
    entropy_gen_number = np.zeros_like(entropy_gen_unit)
    comsol_data.mesh.clear_point_data()

    for idx_snap in tqdm(range(N_SNAPS), total = N_SNAPS):
        param_df = pd.read_csv(param_files[idx_snap], index_col = 0)
        param_df['quantity_pint'] = param_df[param_df.columns[-1]].apply(lambda x : safe_parse_quantity(x))
        lambda_therm = (1 - param_df.loc['host_phi', "quantity_pint"]) * param_df.loc['host_lambda', "quantity_pint"] + \
                            param_df.loc['host_phi', "quantity_pint"] * (4.2 * ureg.watt / (ureg.meter * ureg.kelvin))
        t0      = 0.5 * (param_df.loc["T_h", "quantity_pint"] + param_df.loc["T_c", "quantity_pint"])
        delta_T = (param_df.loc['T_h', "quantity_pint"]  - param_df.loc["T_c", "quantity_pint"])
                    
        for idx_time in range(N_TIME_STEPS):
            
            if IS_USE_TGRAD:
                z_points = comsol_data.mesh.points[:, -1] * ureg.meter
                tgrad = (param_df.loc["T_c", "quantity_pint"] - (delta_T / param_df.loc["H", "quantity_pint"] * z_points)).to('K').magnitude
                temp_data = temperatures_diff[idx_snap, idx_time, :]
            else:
                temp_data = temperatures[idx_snap, idx_time, :]
            
            entrop_gener , entrpy_num = calculate_thermal_entropy_generation(comsol_data.mesh,
                                                                            temp_data,
                                                                            lambda_therm, t0, delta_T)
            entropy_gen_unit[idx_snap, idx_time] = entrop_gener.magnitude
            entropy_gen_number[idx_snap, idx_time] = entrpy_num.magnitude

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:02<00:00,  7.75it/s]


In [14]:
if IS_LOAD_VTU:
    vtu_files = sorted([path for path in import_folder.iterdir() if (path.suffix in [".vtu"] and DATA_TYPE.lower() in path.stem.lower())])
    N_SNAPS = len(vtu_files)
    N_TIME_STEPS = len(COMSOL_VTU( vtu_files[0]).times)
    entropy_gen_unit = np.zeros((N_SNAPS, N_TIME_STEPS))
    entropy_gen_number = np.zeros_like(entropy_gen_unit)
    comsol_data.mesh.clear_point_data()

    for idx_snap, vtu_file in tqdm(enumerate(vtu_files), total = len(vtu_files)): # TODO: Include TQDM
            comsol_data = COMSOL_VTU(vtu_file)
            temperatures = comsol_data.get_array("Temperature")
            comsol_data.mesh.clear_data()
            param_df = pd.read_csv(param_files[idx_snap], index_col = 0)
            ureg = pint.UnitRegistry()
            param_df['quantity_pint'] = param_df[param_df.columns[-1]].apply(lambda x : safe_parse_quantity(x, ureg))
            lambda_therm = (1 - param_df.loc['host_phi', "quantity_pint"]) * param_df.loc['host_lambda', "quantity_pint"] + \
                                param_df.loc['host_phi', "quantity_pint"] * (4.2 * ureg.watt / (ureg.meter * ureg.kelvin))
            t0      = 0.5 * (param_df.loc["T_h", "quantity_pint"] + param_df.loc["T_c", "quantity_pint"])
            delta_T = (param_df.loc['T_h', "quantity_pint"]  - param_df.loc["T_c", "quantity_pint"])
            

            for idx_time, time_key in enumerate(comsol_data.times.keys()):
                entrop_gener , entrpy_num = calculate_thermal_entropy_generation(comsol_data.mesh,
                                                                                temperatures[idx_time, :],
                                                                                lambda_therm, t0, delta_T, ureg = ureg)
                entropy_gen_unit[idx_snap, idx_time] = entrop_gener.magnitude
                entropy_gen_number[idx_snap, idx_time] = entrpy_num.magnitude

In [15]:
np.save(export_folder / f"{DATA_TYPE}_entropy_gen_unit.npy", entropy_gen_unit)
np.save(export_folder / f"{DATA_TYPE}_entropy_gen_number.npy", entropy_gen_number)

### Plot entropy generation

In [16]:
# comsol_data_ref  = COMSOL_VTU( ROOT / "data" / PARAMETER_SPACE / "Training" / f"{DATA_TYPE}_000.vtu")
fig = go.Figure()

for idx_snap, entropies in enumerate(entropy_gen_unit):
    fig.add_trace(go.Scatter(x=list(comsol_data_ref.times.values()),
                             y=entropies,
                                mode='lines',
                                name=f"{idx_snap:03d}",
                                opacity=0.4,
                            line=dict(color=colors[idx_snap])
                ))
    
fig.update_layout(
    xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
    yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
    showlegend=False,        # Hide the legend
    title=f"Entropy generation - {DATA_TYPE} Snapshots - Parameter Space {PARAMETER_SPACE}",       # Optional: add a title
    xaxis_title="Time [s]",    # Optional: label for x-axis
    yaxis_title="Entropy [W/K]"     # Optional: label for y-axis
)
if IS_EXPORT:
    fig.write_image(export_folder / f"{DATA_TYPE}_EntropyGeneration.png")
    fig.write_html(export_folder / f"{DATA_TYPE}_EntropyGeneration.html")
fig.show()

### Plot entropy generation number

In [17]:
fig = go.Figure()

for idx_snap, entropies in enumerate(entropy_gen_number):
    fig.add_trace(go.Scatter(x=list(comsol_data_ref.times.values()),
                             y=entropies,
                                mode='lines',
                                name=f"{idx_snap:03d}",
                                opacity=0.4,
                            line=dict(color=colors[idx_snap])
                ))
    
fig.update_layout(
    xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
    yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
    showlegend=False,        # Hide the legend
    title=f"Entropy generation number - {DATA_TYPE} Snapshots - Parameter Space {PARAMETER_SPACE}",       # Optional: add a title
    xaxis_title="Time [s]",    # Optional: label for x-axis
    yaxis_title="Entropy [-]"     # Optional: label for y-axis
)
if IS_EXPORT:
    fig.write_image(export_folder/ f"{DATA_TYPE}_EntropyGenerationNumber.png")
    fig.write_html(export_folder/ f"{DATA_TYPE}_EntropyGenerationNumber.html")
fig.show()